In [ ]:
import os
import qp
import qp_flexzboost
from flexcode.basis_functions import BasisCoefs
import matplotlib.pyplot as plt
import numpy as np

print(qp_flexzboost.__file__)
print(qp.__file__)

In [ ]:
# Retrieve some real world example coefficients (i.e. weights) that are used for testing.
qp_flexzboost.FlexzboostGen.make_test_data()
coefs = qp_flexzboost.FlexzboostGen.test_data['weights']

In [ ]:
# Demonstrate the creation of a `FlexCode.BasisCoefs` object.
basis_coefficients = BasisCoefs(coefs,
                                basis_system='cosine',
                                z_min=0.0,
                                z_max=3.0,
                                bump_threshold=0.1,
                                sharpen_alpha=1.2)

In [ ]:
# Just an example to show how the basis_coefficient.evaluate method works.
# Notice that it doesn't take a simple 1D x array.
x = np.linspace(0,3,100)
print(x.shape)
x_vals = x.reshape(-1,1)
print(x_vals.shape)
y_vals = basis_coefficients.evaluate(x_vals)

# I expected this to work, namely passing an array with size (10, 100) to the evaluate method. 
# The goal is to show that evaluate can handle different x values per PDF - even though 
# here it would just be repeating the same x values 10 times. There might be a bug
# in the Flexcode code around basis_functions.py:44

# xx_vals = np.tile(x, [10, 1])
# print(xx_vals.shape)
# yy_vals = basis_coefficients.evaluate(xx_vals)

In [ ]:
#Demonstrate building the ensemble using either the BasisCoefs object or the individual properties of the BasisCoef object
fzb = qp.Ensemble(qp_flexzboost.flexzboost_create_from_basis_coef_object, data=dict(weights=coefs, basis_coefficients_object=basis_coefficients))
fzb2 = qp.Ensemble(qp_flexzboost.flexzboost, 
                   data=dict(weights=coefs, basis_system_enum_value=1, z_min=0.0, z_max=3.0, bump_threshold=0.1, sharpen_alpha=1.2))
print(fzb.dist)
print(fzb2.dist)

In [ ]:
# Demonstrate that the output PDF values are the same regardless of whether the ensemble
# is constructed with a BasisCoef or with the individual properties of the BasisCoef
pdf_id = 6
x = np.linspace(0,3,100)

print(np.sum(fzb[pdf_id].pdf(x) - fzb2[pdf_id].pdf(x)))

In [ ]:
# Demonstrate simple plotting of a single PDF in the ensemble
qp.plotting.plot_native(fzb[pdf_id], xlim=[0,3])

In [ ]:
# Demonstrate that CDFs work as expected

# A single CDF from the ensemble
plt.plot(x, np.squeeze(fzb[pdf_id].cdf(x)), linewidth=5)

# Calculate the CDF for all distributions in the ensemble, and then select one
cdfs = fzb.cdf(x)
cdfs[pdf_id]
plt.plot(x, cdfs[pdf_id], linestyle='--' )

In [ ]:
# Demonstrate that building tables for output to disk works as expected.
tabs = fzb.build_tables()
print(tabs.keys())
print("Meta Data")
print(tabs['meta'])
print()
print("Object Data")
print(tabs['data'])

In [ ]:
# Demonstrate that the ensemble can be written to disk, and read back in with no loss of information

output_fits = "test_output.fits"
output_hdf5 = "test_output.hdf5"

# delete the files if they already exist
try:
    os.unlink(output_hdf5)
    os.unlink(output_fits)
except FileNotFoundError:
    pass

# write out the files
fzb.write_to(output_hdf5)
print(".hdf5 file size is:", os.path.getsize(output_hdf5), "bytes")
fzb.write_to(output_fits)
print(".fits file size is:", os.path.getsize(output_fits), "bytes")

# read the files back in
fzb_reread_hdf5 = qp.read(output_hdf5)
fzb_reread_fits = qp.read(output_fits)

# Show that the number of PDFs is the same after reading in the files
print("Initial number of pdfs:", fzb.npdf)
print("Recovered number of pdfs, hdf5:", fzb_reread_hdf5.npdf)
print("Recovered number of pdfs, fits:", fzb_reread_fits.npdf)

# Show that the plots for a given PDF are the same
_, ax = qp.plotting.plot_native(fzb_reread_hdf5[pdf_id], xlim=[0,3])
qp.plotting.plot_native(fzb_reread_fits[pdf_id], axes=ax)

# Show that nothing has been lost in the file type storage methods
pdf_hdf5 = fzb_reread_hdf5[pdf_id].pdf(x_vals)
pdf_fits = fzb_reread_fits[pdf_id].pdf(x_vals)
print("Total difference in file storage types:", sum((pdf_fits-pdf_hdf5)**2))

# show that all the parameters to define the BasisCoef object have been recovered
print("Initial bump_threshold:", fzb.dist.basis_coefficients.bump_threshold)
print("Recovered fits bump_threshold:", fzb_reread_fits.dist.basis_coefficients.bump_threshold)
print("Recovered hdf5 bump_threshold:", fzb_reread_hdf5.dist.basis_coefficients.bump_threshold)

# delete the output files that were written
try:
    os.unlink(output_hdf5)
    os.unlink(output_fits)
except FileNotFoundError:
    pass

In [ ]:
# Demonstrate that the Flexzboost parameterization of the data can be converted
# to other representations. For instance here, an interpolated grid.
ens_interp = fzb.convert_to(qp.interp_gen, xvals=np.linspace(0,3,100))

# Plot interpolated PDF (thick line)
qp.plotting.plot_native(ens_interp[pdf_id], xlim=[0,3], linewidth=5)

# Plot original, Flexzboost PDF (dashed line)
plt.plot(x, np.squeeze(fzb[pdf_id].pdf(x)), linestyle='--')

In [ ]:
#Demonstrating that the bump threshold and sharpening alpha parameters can be changed without rerunning the model.
# Set the bump threshold and sharpening parameters to the original values
fzb.dist.bump_threshold = 0.1
fzb.dist.sharpen_alpha = 1.2

# Plot original, Flexzboost PDF (dashed line)
plt.plot(x, np.squeeze(fzb[pdf_id].pdf(x)), linewidth=5, label='Non-None bump and sharpen parameters')

# remove the bump threshold and sharpening parameters
fzb.dist.bump_threshold = None
fzb.dist.sharpen_alpha = None

plt.plot(x, np.squeeze(fzb[pdf_id].pdf(x)), label='bump_threshold=sharpen_alpha=None')
plt.legend()